In [1]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import itertools
import copy
from matplotlib.ticker import FormatStrFormatter
from scipy import stats
import seaborn as sns
from sklearn.metrics import r2_score
from sklearn.preprocessing import scale
from sklearn.preprocessing import robust_scale
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from tabulate import tabulate
from sklearn.decomposition import PCA
from ddot import Ontology

pd.options.mode.chained_assignment = None  # default='warn'

In [2]:
sns.set()
sns.set_style("white")
sns.set_style("ticks", {"xtick.major.size":8, "ytick.major.size":8})
sns.axes_style("whitegrid")
sns.set_palette("muted")
sns.color_palette("muted")

[(0.2823529411764706, 0.47058823529411764, 0.8156862745098039),
 (0.9333333333333333, 0.5215686274509804, 0.2901960784313726),
 (0.41568627450980394, 0.8, 0.39215686274509803),
 (0.8392156862745098, 0.37254901960784315, 0.37254901960784315),
 (0.5843137254901961, 0.4235294117647059, 0.7058823529411765),
 (0.5490196078431373, 0.3803921568627451, 0.23529411764705882),
 (0.8627450980392157, 0.49411764705882355, 0.7529411764705882),
 (0.4745098039215686, 0.4745098039215686, 0.4745098039215686),
 (0.8352941176470589, 0.7333333333333333, 0.403921568627451),
 (0.5098039215686274, 0.7764705882352941, 0.8862745098039215)]

In [3]:
plt.rcParams['svg.fonttype'] = 'none'

SMALL_SIZE = 12
MEDIUM_SIZE = 16
BIGGER_SIZE = 22

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

In [4]:
def read_file(filename, delim='\n'):
	rfile = open(filename, 'r')
	item_list = rfile.read().strip().split(delim)
	rfile.close()
	return item_list

In [5]:
ont = 'ctg'
dataset = 'av'

drug_info = pd.read_csv("../data/master_druglist_smiles_final.csv")[['name', 'isomeric_smiles', 'ctrp2_name']]
drug_info['name'] = drug_info['name'].str.replace(' ','-')
drug_info['name'] = drug_info['name'].str.replace('"','')

drug_smiles_map = dict(zip(drug_info.isomeric_smiles, drug_info.name))

drug_name_map = dict(zip(drug_info.name, drug_info.isomeric_smiles))

/cellar/users/asinghal/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (4,5,6,7,8,9,10,11,12,13,14,15,16,18,20,21,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [6]:
task = 'RS'
    
task_file='../data/training_files_' + dataset + '/task_list_' + task + '.txt'
rs_drugs = read_file(task_file)

In [7]:
control_drugs_names = ['Ursolic-acid', 'Afatinib', 'Gefitinib']
control_drugs = [drug_name_map[d] for d in control_drugs_names]

In [8]:
all_combinations = []
for i in range(1, len(control_drugs) + 1):
    ctrl_list_list = [list(c) for c in itertools.combinations(control_drugs, i)]
    rs_list_list = [list(c) for c in itertools.combinations(rs_drugs, len(rs_drugs) - i)]
    for rs_list in rs_list_list:
        for cl in ctrl_list_list:
            dl = copy.deepcopy(rs_list)
            dl.extend(cl)
            all_combinations.append(dl)

In [12]:
for i, dl in enumerate(all_combinations):
    task_file='../data/training_files_' + dataset + '/control/task_list_' + task + '_' + str(i+1) + '.txt'
    with open(task_file, 'w') as fp:
        for drug in dl:
            fp.write("%s\n" % drug)